# QC-Amp: Colour Factors for Table 1 Diagrams



This notebook uses the `qc_amp` library to compute quantum chromodynamics (QCD) colour factors
for the diagrams listed in an abstracted **Table 1**.

- We first recall the definition of the colour factor and the normalisation.
- We then construct quantum circuits for each diagram using the `qc_amp.circuits` API.
- Finally, we evaluate the corresponding colour factors using `qc_amp.colour_factors`.

## Physical Background: QCD and Colour

**Quantum Chromodynamics (QCD)** is the gauge theory of the strong interaction.  Unlike QED, where the photon is neutral, in QCD the gauge bosons — *gluons* — carry colour charge themselves.  This leads to a rich group-theoretic structure encoded in the **colour factors** of Feynman diagrams.

### The SU(3) Colour Group

Quarks transform in the **fundamental representation** of the Lie group SU(3), meaning each quark carries one of three colour charges (conventionally labelled red, green, blue).  Mathematically, a quark colour state lives in a 3-dimensional complex vector space $\mathbb{C}^3$.

Gluons transform in the **adjoint representation**, which has dimension $N_c^2 - 1 = 8$ for SU(3).  Each gluon therefore carries a combination of colour and anti-colour.

### Generators and Gell-Mann Matrices

The Lie algebra $\mathfrak{su}(3)$ has 8 generators, usually written $T^a$ with $a = 1, \ldots, 8$.  In the fundamental representation these are $3 \times 3$ Hermitian, traceless matrices related to the **Gell-Mann matrices** $\lambda^a$ by

$$
T^a = \frac{\lambda^a}{2}.
$$

The Gell-Mann matrices satisfy the orthogonality relation

$$
\operatorname{Tr}(\lambda^a \lambda^b) = 2\,\delta^{ab},
$$

which implies

$$
\operatorname{Tr}(T^a T^b) = \frac{1}{2}\,\delta^{ab}.
$$

### Colour Factors in Feynman Diagrams

Every QCD Feynman diagram contains a product of generator matrices $T^a$ at each quark–gluon vertex, contracted over gluon indices.  The **colour factor** $C$ is the result of evaluating this group-theoretic expression, summing over all internal colour indices.

For example, the simplest self-energy diagram — a quark emitting and immediately reabsorbing a gluon — yields

$$
C = \sum_{a=1}^{8} (T^a)_{ij}(T^a)_{jk}\,\delta_{ik} = \sum_a \operatorname{Tr}\bigl[(T^a)^2\bigr].
$$

Using $\operatorname{Tr}(T^a T^b) = \frac{1}{2}\delta^{ab}$ we get

$$
C = \sum_a \frac{1}{2} = \frac{8}{2} = 4.
$$

This is the famous **quadratic Casimir** $C_F = 4/3$ multiplied by $N_c = 3$, i.e.\ $C_F \cdot N_c = 4$.

## 1. Quantum-Circuit Formulation of Colour Factors

The key insight of [Chawdhry & Pellen (2023)](https://arxiv.org/abs/2311.12324) is that **colour factors can be computed on a quantum computer** by encoding the group-theoretic sums into unitary gates.

### Qubit Encoding of Colour States

| Particle | Hilbert space dimension | Qubits required |
|----------|------------------------|-----------------|
| Quark colour | $N_c = 3$ | 2 (states $|00\rangle, |01\rangle, |10\rangle$; $|11\rangle$ unused) |
| Gluon colour | $N_c^2 - 1 = 8$ | 3 (states $|000\rangle$ to $|111\rangle$) |

### The Unitarisation Problem

The generator matrices $T^a$ are Hermitian but **not unitary**.  To implement them in a quantum circuit, the algorithm introduces:

1. **Unitary-adjusted matrices** $\hat\lambda^a$ that *are* unitary and agree with $\lambda^a$ on most entries, but with specific modifications (e.g.\ $\hat\lambda^8 = \mathbb{1}$).
2. **Coefficients** $\mu(a,i)$ that capture the ratio between the original and adjusted matrices:
   $$
   \mu(a,i)\,\hat\lambda^a\,|i\rangle = \frac{1}{2}\lambda^a\,|i\rangle = T^a\,|i\rangle.
   $$
3. A **unitarisation register** $U$ whose amplitude encodes the product of $\mu$ coefficients along a diagram.

### The Q Gate (Quark–Gluon Vertex)

The fundamental building block is the **Q gate**, which implements one quark–gluon vertex:

$$
Q = \bigl(\Lambda \otimes \mathbb{1}_U\bigr)\; M \; \bigl(\mathbb{1}_g \otimes \mathbb{1}_q \otimes A\bigr),
$$

where

- $A$ is the **increment operator** on the unitarisation register,
- $M$ applies controlled-$B(\mu)$ gates that transfer the $\mu$ coefficients into the $U$-register amplitude,
- $\Lambda$ applies the controlled unitary $\hat\lambda^a$ on the quark register, conditioned on the gluon colour $|a-1\rangle$.

### Extracting the Colour Factor

After building the full diagram circuit (preparation, vertices, inverse preparation), the colour factor is obtained from the overlap with the all-zero state:

$$
C = N \,\langle \Omega | \psi_{\text{final}} \rangle,
$$

where the **normalisation factor** accounts for the uniform superpositions in the colour registers:

$$
N = N_c^{\,n_q}\,(N_c^2 - 1)^{n_g}, \qquad N_c = 3.
$$

For a single quark line ($n_q = 1$) and one gluon ($n_g = 1$), we have $N = 3 \times 8 = 24$.

## 2. Imports and helper functions



We import the QC-Amp library and define a small helper to run a circuit and report the colour factor and normalisation.

In [ ]:
from qc_amp.circuits import quark_emission_absorption

from qc_amp.colour_factors import compute_colour_factor_detailed



def compute_and_report_colour_factor(circuit, n_quarks: int, n_gluons: int, N_c: int = 3):

    """Run the colour-factor computation and return a small dict summary."""

    C, amp_omega, N = compute_colour_factor_detailed(

        circuit,

        n_quarks=n_quarks,

        n_gluons=n_gluons,

        N_c=N_c,

    )

    return {

        'colour_factor': C,

        'overlap_omega': amp_omega,

        'normalisation_N': N,

    }



N_c = 3

## 3. Diagram D1: Quark Self-Energy (Emission and Absorption)

### Physical Interpretation

Diagram D1 is the simplest QCD self-energy correction: a quark propagates, **emits** a virtual gluon, then **reabsorbs** it before continuing.  In momentum space this contributes to the quark self-energy $\Sigma(p)$; the colour factor multiplies the kinematic integral.

```
    q ───●───────●─── q
         ╲     ╱
          ╲   ╱
           ╲ ╱   gluon (colour a)
```

### Colour-Algebra Calculation

At the emission vertex the quark colour changes as $(T^a)_{ij}$; at the absorption vertex it changes back as $(T^a)_{jk}$.  External quarks are traced over, so:

$$
C_{\text{D1}} = \sum_{a=1}^{8} \sum_{i,j,k=1}^{3} (T^a)_{ij}\,(T^a)_{jk}\,\delta_{ik}
= \sum_a \operatorname{Tr}\bigl[(T^a)^2\bigr].
$$

Using $\operatorname{Tr}(T^a T^b) = \tfrac{1}{2}\delta^{ab}$:

$$
C_{\text{D1}} = \sum_{a=1}^{8} \frac{1}{2} = \frac{8}{2} = 4.
$$

### Relation to the Casimir Invariant

The quadratic Casimir of the fundamental representation is

$$
C_F = \frac{N_c^2 - 1}{2 N_c} = \frac{8}{6} = \frac{4}{3}.
$$

The colour factor here equals $C_F \cdot N_c = \tfrac{4}{3} \times 3 = 4$, which is what we expect from the trace formula.

### Quantum-Circuit Implementation

The circuit `quark_emission_absorption(n_vertices=2)` implements this diagram:

1. Prepare the gluon register in an equal superposition of 8 colours (H⊗3).
2. Prepare the quark–antiquark pair in a colour-singlet state $\tfrac{1}{\sqrt{3}}\sum_k |k\rangle_q |k\rangle_{\bar q}$.
3. Apply **two Q gates** on the quark register (emission then absorption).
4. Apply the inverse preparations to project back onto the colour-singlet/gluon-vacuum state.

The final overlap $\langle\Omega|\psi\rangle$ times $N = 24$ should give **C = 4**.

In [ ]:
# Build the circuit for D1: quark emission and absorption

circ_D1 = quark_emission_absorption(n_vertices=2)

circ_D1.draw(output='mpl')

In [ ]:
# Compute the colour factor for D1

result_D1 = compute_and_report_colour_factor(

    circ_D1,

    n_quarks=1,

    n_gluons=1,

    N_c=N_c,

)

result_D1